# Miniproyecto 3

### David Soto 17551 & Miguel Valle 17102

In [1]:
### Librerias a utilizar para la simulacion
import numpy as np
import random

### Función generadora de tiempos de estadía/salida

In [2]:
# Funcion para calcular el proximo tiempo de llegada
def tiempoProximaSalida(t, lamda_max, homo):
    if homo:
        return t - ((1/lamda_max)*np.log(random.random()))
    else:
        while True:
            u = random.random()
            t = t - ((1/lamda_max)*np.log(u))
            u = random.random()
            if u <= np.random.poisson(t)/lamda_max:
                return t

### Funcion simuladora de eventos

In [3]:
### Funcion que determina cómo procede el evento segun el tiempo dela simulacion
def evento(t, n, Na, C, busyTime, A, D, NT, ta, td, ids, lamda_max, no_cpus, speed):
    ### Si la proxima peticion es el ultimo registrado en el sistema, entonces hay un servidor libre que lo puede atender
    if ta == min(ta,min(td)): 
        t = ta
        Na = Na + 1
        ta = tiempoProximaSalida(t, lamda_max, True)
        A[Na] = t
        ### Si la cantidad de peticiones en el sistema es menor a la cantidad de cpus
        ### es porque la proxima peticion puede ser procesada por un servidor
        if n < no_cpus:
            for i in range(no_cpus):
                if ids[i] == 0:
                    ids[i] = Na
                    NT = np.append(NT, t - A[Na])
                    td[i] = t + np.random.exponential(speed)
                    busyTime[i] = busyTime[i] + td[i] - t
                    break
        n = n + 1
    else:
        ### Esto ocurre en caso de que no haya un servidor desocupado al momento de llegar
        ti = np.argmin(td)
        t = td[ti]
        C[ti] = C[ti] + 1
        D[ids[ti]] = t
        ### En caso la cantidad de clientes sea menor o igual a la cantidad de servidores
        ### Se libera un CPU
        if n <= no_cpus:
            ### Se elimina a la ultima peticion del ID register
            ids[ti] = 0
            td[ti] = np.infty
        ### Si todavia hay mas clientes por atender
        else: 
            indice = max(ids) + 1
            ### Se actualiza el indice
            ids[ti] = indice
            NT = np.append(NT , t - A[indice])
            td[ti] = t + np.random.exponential(speed)
            busyTime[ti] = busyTime[ti] + td[ti] - t
        n = n - 1
        
    ### Retornamos las variables para actualizacion y seguir el proceso de la simulacion 
    return t, n, Na, C, busyTime, A, D, NT, ta, td, ids

### Funcion simuladora del procesamiento de peticiones

In [4]:
### Funcion para realizar la simulacion
def simular(duracion_simulacion, lamda_max, speed,no_cpus):
    ### Variable de tiempo de simulacion
    t = 0
    
    # Variables contadores
    Na = 0
    C = np.zeros(no_cpus) 
    busyTime = np.zeros(no_cpus)
    
    ### Variable estado del sistema
    n = 0

    ### Variables para el control de datos
    A = {}
    D = {}
    NT = []
    
    # Variables de eventos de tiempos de llegada y salida
    ta = tiempoProximaSalida(t,lamda_max, True)
    td = np.zeros(no_cpus) + np.infty
    ids = np.zeros(no_cpus)
    
    # Se realiza la simulacion hasta que se llegue al tiempo final de esta
    while t < duracion_simulacion:
        ### Realizamos la simulacion de un evento al timepo t
        t, n, Na, C, busyTime, A, D, NT, ta, td, ids = evento(t, n, Na, C, busyTime, A, D, NT, ta, td, ids, lamda_max, no_cpus, speed)
        
    ### Mostramos los resultados de las variables de la somulacion
    print("Solicitudes atendidas: ", C)
    print("Tiempos de ocupacion de servidores: ", busyTime)
    print("Tiempos libres de servidores: ", np.maximum(np.ones(no_cpus)*duracion_simulacion - busyTime,0))
    print("Tiempo de solicitudes en cola: ", np.round(sum(NT),5))
    print("Promedio de solicitudes en cola:", np.round(np.mean(NT),5))
    print("Tiempo de salida de la ultima solicitud atendida: ", np.round(td[-1],8))

## Task 1 - Simulacion con 2400 solicitudes por minuto

### Simulacion - 2400 solicitudes por minuto -  Gorilla Megacomputing

In [5]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(100*60)
cpus = 1

### Procesamiento de cada simulacion
print("Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de 0.00017 solicitudes/s
Solicitudes atendidas:  [143995.]
Tiempos de ocupacion de servidores:  [24.06576165]
Tiempos libres de servidores:  [35.93423835]
Tiempo de solicitudes en cola:  16.09402
Promedio de solicitudes en cola: 0.00011
Tiempo de salida de la ultima solicitud atendida:  60.0006875


### Simulacion - 2400 solicitudes por minuto - Ants smart computing

In [6]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 10

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [28918. 26510. 23647. 20320. 16428. 11978.  8045.  4899.  2659.  1359.]
Tiempos de ocupacion de servidores:  [48.07454029 44.5148567  39.91501882 33.97314639 27.22216822 19.83611898
 13.54708189  8.12224756  4.46337866  2.23912107]
Tiempos libres de servidores:  [11.92545971 15.4851433  20.08498118 26.02685361 32.77783178 40.16388102
 46.45291811 51.87775244 55.53662134 57.76087893]
Tiempo de solicitudes en cola:  0.35453
Promedio de solicitudes en cola: 0.0
Tiempo de salida de la ultima solicitud atendida:  inf


## Task 2 - Determinacion de numero de servidores para no tener cola con Ants con 2400 solicitudes por minuto

### Simulacion - 2400 solicitudes por minuto - Ants smart computing

In [7]:
### Variables constantes para la simulacion
solicitudes_max = 2400
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 15

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)

Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [2.8917e+04 2.6440e+04 2.3802e+04 2.0227e+04 1.5994e+04 1.1765e+04
 7.7440e+03 4.6490e+03 2.4010e+03 1.1290e+03 4.5000e+02 1.7100e+02
 6.2000e+01 1.9000e+01 6.0000e+00]
Tiempos de ocupacion de servidores:  [4.80814756e+01 4.43904626e+01 3.94421775e+01 3.34614236e+01
 2.69380140e+01 1.95297441e+01 1.29869741e+01 7.63664260e+00
 4.13286274e+00 1.93851615e+00 7.40923587e-01 2.95018338e-01
 1.35990717e-01 2.51758228e-02 6.66580164e-03]
Tiempos libres de servidores:  [11.91852441 15.60953739 20.55782249 26.53857643 33.061986   40.47025589
 47.01302589 52.3633574  55.86713726 58.06148385 59.25907641 59.70498166
 59.86400928 59.97482418 59.9933342 ]
Tiempo de solicitudes en cola:  0.00058
Promedio de solicitudes en cola: 0.0
Tiempo de salida de la ultima solicitud atendida:  inf


## Task 3 - Simulacion con 6000 solicitudes por minuto

### Simulacion - 6000 solicitudes por minuto - Gorilla Megacomputing

In [31]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(100*60)
cpus = 1

### Procesamiento de cada simulacion
print("Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Gorilla Megacomputing con 1 CPU y una velocidad de 0.00017 solicitudes/s
Solicitudes atendidas:  [360060.]
Tiempos de ocupacion de servidores:  [59.87849867]
Tiempos libres de servidores:  [0.12150133]
Tiempo de solicitudes en cola:  14020.08068
Promedio de solicitudes en cola: 0.03894
Tiempo de salida de la ultima solicitud atendida:  60.00018481


### Simulacion - 6000 solicitudes por minuto - Ants smart computing

In [35]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 10

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)


Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [36081. 36167. 36245. 35778. 35816. 36027. 36081. 35826. 35802. 36179.]
Tiempos de ocupacion de servidores:  [59.98109641 59.97180608 59.972499   59.9666373  59.95565571 59.95514574
 59.95060128 59.93156736 59.91387123 59.91538105]
Tiempos libres de servidores:  [0.01890359 0.02819392 0.027501   0.0333627  0.04434429 0.04485426
 0.04939872 0.06843264 0.08612877 0.08461895]
Tiempo de solicitudes en cola:  23807.90352
Promedio de solicitudes en cola: 0.06613
Tiempo de salida de la ultima solicitud atendida:  60.00063864


## Task 4 - Determinacion de numero de servidores para no tener cola con Ants con 6000 solicitudes por minuto

### Simulacion - 6000 solicitudes por minuto - Ants smart computing

In [37]:
### Variables constantes para la simulacion
solicitudes_max = 6000
duracionSimulacion = 60
speed = 1/(10*60)
cpus = 25

### Procesamiento de cada simulacion
print("Simulacion para Ants smart computing con 10 CPU y una velocidad de", round(speed, 5), "solicitudes/s")
simular(duracionSimulacion,solicitudes_max, speed, cpus)

Simulacion para Ants smart computing con 10 CPU y una velocidad de 0.00167 solicitudes/s
Solicitudes atendidas:  [3.2940e+04 3.2410e+04 3.1745e+04 3.0897e+04 2.9788e+04 2.8480e+04
 2.7248e+04 2.5314e+04 2.3089e+04 2.1020e+04 1.8541e+04 1.5656e+04
 1.2799e+04 9.9750e+03 7.4440e+03 5.2390e+03 3.4850e+03 2.1310e+03
 1.2610e+03 6.8400e+02 3.3100e+02 1.6500e+02 8.7000e+01 4.2000e+01
 1.0000e+01]
Tiempos de ocupacion de servidores:  [5.45109307e+01 5.36059176e+01 5.25215423e+01 5.11979420e+01
 4.94982572e+01 4.77764591e+01 4.52358128e+01 4.24636803e+01
 3.90923720e+01 3.54171098e+01 3.08277195e+01 2.62767413e+01
 2.13849459e+01 1.67686187e+01 1.26028362e+01 8.72890922e+00
 5.76869787e+00 3.62871034e+00 2.06098072e+00 1.20738246e+00
 5.08863349e-01 2.65755751e-01 1.47980534e-01 5.77510270e-02
 1.30350918e-02]
Tiempos libres de servidores:  [ 5.48906929  6.39408244  7.47845767  8.80205802 10.50174278 12.22354085
 14.76418721 17.53631975 20.90762801 24.58289016 29.17228045 33.72325866
 38.61505

## Task 5 - Recomendacion a la junta directiva

Dado que se observa una mejora clara en el tiempo de espera para las solicitudes hechas, se recomienda alquilar 15 servidores para un numero de solicitudes de 2400, y de alquilar 30 servidores para 6000 solicitudes, ya que aunque alquilar más servidores podría reducir aún más el tiempo de espera, no se estaran utilizando todos cuando no se tenga una cantidad de solicitudes muy elevada, por lo cual sería un desperdicio del presupuesto. Es preferible tener menos servidores, pero que todos se estén aprovechando durante un flujo normal de solicitudes realizada, tomando como línea base 2400 y límite superior valores cercanos a 6000. Por lo tanto la cantidad de servidores que se recomienda alquilar se encuentra en un rango entre 15 y 30 servidores.